<a href="https://colab.research.google.com/github/jkroberts1/CS598_DLH-Spring24-Group47/blob/main/CS_598_DLH_Project_Evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/My Drive/CS598_DLH/Dataset2

Mounted at /content/drive
/content/drive/My Drive/CS598_DLH/Dataset2


#Model Building

This section will go over the implementation of the model

Our Citation for the original paper is in the References section at [2].

Original code is here: https://github.com/yejinjkim/synergy-transfer

In [2]:
import os
import torch
import numpy as np
import pandas as pd
import pdb
import time
import pickle
import logging
import matplotlib.pyplot as plt
from utilities import Mapping


import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [3]:
data_path=''
save_path=''

bsz=128
cuda=True
device=3
#torch.cuda.set_device(device)

num_gene_compressed_drug=64
num_gene_compressed_cell=128

#isClassification=True #False for regression task
syn_threshold=30
ri_threshold=50

log_interval=100
epochs=10

##Load data


In [4]:
#drug pair, cell, scoers
df=pickle.load(open(data_path+'summary_mean.p', 'rb'))
codes=pickle.load(open(data_path+'codes.p', 'rb'))

In [5]:
df.head()



,drug_row,drug_col,cell_line_name,study_id,ri_row,ri_col,synergy_loewe
0,2617,3003,0,4,-21.079400,17.392589,4.436431
1,1515,3003,0,4,-4.051616,17.392589,10.755529
2,904,1413,1,11,-71.949000,9.755000,-6.188270
3,904,3003,0,4,-9.231525,17.392589,0.739056
4,63,1413,1,11,-12.272000,15.133000,-18.680123


In [6]:
# Drug's external features
drug_features=pickle.load(open(data_path+'drug_features.p', 'rb'))

In [7]:
# Cell's external features
cell_features=pickle.load(open(data_path+'cell_features.p', 'rb'))

In [8]:
num_celllines= len(codes['cell'].idx2item)
num_drugs=len(codes['drugs'].idx2item)

num_genes = len(codes['gene'].idx2item)
num_tissue = len(codes['tissue'].idx2item)
num_disease = len(codes['disease'].idx2item)

num_drug_fp=len(drug_features.loc[0,'fps'])
max_drug_sm_len = drug_features['smiles'].apply(lambda x: len(x)).max()

#Gene compression
This section will go over the dataloaders that cover the cell lines compression for the model training and the dataloaders that cover the drug target compression.

##Dataset and Dataloader

In [9]:
class DrugTargetDataset(Dataset):
    def __init__(self, drug_features):
        self.drug_features = drug_features
    def __len__(self):
        return len(self.drug_features)
    def __getitem__(self,idx):
            gene_ids=self.drug_features.loc[idx, 'gene_id']
            genes=np.zeros(num_genes)
            genes[gene_ids]=1

            return genes


class CellGeneDataset(Dataset):
    def __init__(self, cell_features):
        self.cell_features = cell_features
    def __len__(self):
        return len(self.cell_features)
    def __getitem__(self,idx):
            gene_ids=self.cell_features.loc[idx,'gene_id']
            genes = np.zeros(num_genes)
            for key,value in gene_ids.items():
                genes[key]=value

            return genes


In [10]:
#Two layers of fully connected layers
class FC2(nn.Module):
    def __init__(self, in_features, out_features, dropout):
        super(FC2, self).__init__()

        self.bn = nn.BatchNorm1d(in_features)
        self.fc1 = nn.Linear(in_features, int(in_features/2))
        self.fc2 = nn.Linear(int(in_features/2),out_features)
        self.dropout= nn.Dropout(dropout)

    def forward(self, x):
        x = self.bn(x)
        x = self.dropout(x)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)

        return x

##Compress gene features
Here is where the compression happens. For cell lines COSMIC and CCLE are combined.

For drug targets, DrugBank, NIH-LINC, and TTD are combined.

In [11]:
class GeneCompressor(nn.Module):
    def __init__(self, num_in, num_out, dropout=0.1):
        super(GeneCompressor, self).__init__()
        self.dropout=dropout
        self.encoder=nn.Linear(num_in, num_out)
        self.decoder=nn.Linear(num_out,num_in)

    def _encoder(self,x):
        return F.dropout(F.relu(self.encoder(x)), self.dropout, training=self.training)

    def _decoder(self,x):
        return F.dropout(self.decoder(x), self.dropout, training=self.training)

    def forward(self, x):
        x=self._encoder(x)
        x=self._decoder(x)
        return x

In [12]:
def geneCompressing(data_loader,num_gene_compressed, noise_weight=0.2, epochs=20, log_interval=10 ):
    #model
    geneCompressor=GeneCompressor(num_genes, num_out=num_gene_compressed, dropout=0.1)
    if cuda:
        geneCompressor=geneCompressor.cuda()
    criterion=nn.MSELoss()
    optimizer=optim.Adam(geneCompressor.parameters())
    for epoch in range(1,epochs+1):
        #train
        geneCompressor.train()
        total_loss=0
        start_time=time.time()
        for iteration, gene in enumerate(data_loader):
            gene=Variable(gene).float()
            noise=noise_weight*torch.randn(gene.shape)

            if cuda:
                gene=gene.cuda()
                noise=noise.cuda()
            optimizer.zero_grad()
            output=geneCompressor(gene+noise)
            loss=criterion(output,gene)
            loss.backward()
            optimizer.step()
            total_loss += loss.data
            if iteration % log_interval == 0 and iteration > 0:
                cur_loss = total_loss.item() / log_interval
                elapsed = time.time() - start_time
                print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | loss {:8.5f}'.format(epoch, iteration, int(len(data_loader)/bsz), elapsed * 1000/log_interval, cur_loss))
                total_loss = 0
                start_time = time.time()
#         #test
#         geneCompressor.eval()
#         total_loss=0
#         start_time=time.time()
#         with torch.no_grad():
#             for iteration, gene in enumerate(test_data_loader):
#                 gene=Variable(gene).float()
#                 if cuda:
#                     gene=gene.cuda(device)
#                 output=geneCompressor(gene)
#                 loss=criterion(output,gene)
#                 total_loss += loss.data
#             print(total_loss.item()/iteration)
    return geneCompressor

In [ ]:

#drug's target gene data
drugGeneDataset=DrugTargetDataset(drug_features)
drugGeneDataset_loader = DataLoader(drugGeneDataset, batch_size=64, shuffle=True)
#learn
drugGeneCompressor=geneCompressing(drugGeneDataset_loader, num_gene_compressed_drug)
#save
drugGeneCompressor.eval()
drugGeneCompressed=np.array([drugGeneCompressor.cpu()._encoder(torch.FloatTensor(drugGeneDataset[d])).data.numpy() for d in range(num_drugs)])
torch.save(drugGeneCompressor.state_dict(), data_path+'drugGeneCompressor.p')
pickle.dump(drugGeneCompressed, open(data_path+'drugGeneCompressed.p', 'wb'))

| epoch   1 |    10/    0 batches | ms/batch 99.24 | loss  0.00786
| epoch   1 |    20/    0 batches | ms/batch 24.56 | loss  0.00518
| epoch   1 |    30/    0 batches | ms/batch 26.08 | loss  0.00521
| epoch   1 |    40/    0 batches | ms/batch 24.08 | loss  0.00482
| epoch   2 |    10/    0 batches | ms/batch 24.55 | loss  0.00473
| epoch   2 |    20/    0 batches | ms/batch 24.11 | loss  0.00414
| epoch   2 |    30/    0 batches | ms/batch 23.87 | loss  0.00365
| epoch   2 |    40/    0 batches | ms/batch 23.93 | loss  0.00325
| epoch   3 |    10/    0 batches | ms/batch 26.85 | loss  0.00249
| epoch   3 |    20/    0 batches | ms/batch 22.73 | loss  0.00200
| epoch   3 |    30/    0 batches | ms/batch 24.10 | loss  0.00173
| epoch   3 |    40/    0 batches | ms/batch 22.76 | loss  0.00149
| epoch   4 |    10/    0 batches | ms/batch 26.11 | loss  0.00116
| epoch   4 |    20/    0 batches | ms/batch 23.67 | loss  0.00091
| epoch   4 |    30/    0 batches | ms/batch 22.52 | loss  0.0

In [13]:

drugGeneCompressed=pickle.load(open(data_path+'drugGeneCompressed.p', 'rb'))
drugGeneCompressed=torch.FloatTensor(drugGeneCompressed)

In [ ]:
#cell line's gene expression data
cellGeneDataset=CellGeneDataset(cell_features)
cellGeneDataset_loader = DataLoader(cellGeneDataset, batch_size=64, shuffle=True)
#learn
cellGeneCompressor=geneCompressing(cellGeneDataset_loader,num_gene_compressed_cell, noise_weight=0.01, log_interval=1 )
#save
cellGeneCompressor.eval()
cellGeneCompressed=np.array([cellGeneCompressor.cpu()._encoder(torch.FloatTensor(cellGeneDataset[d])).data.numpy() for d in range(num_celllines)])
torch.save(cellGeneCompressor.state_dict(), data_path+'cellGeneCompressor.p')
pickle.dump(cellGeneCompressed, open(data_path+'cellGeneCompressed.p', 'wb'))

| epoch   1 |     1/    0 batches | ms/batch 414.69 | loss  2.74390
| epoch   2 |     1/    0 batches | ms/batch 422.43 | loss  3.53132
| epoch   3 |     1/    0 batches | ms/batch 416.57 | loss  2.99976
| epoch   4 |     1/    0 batches | ms/batch 406.06 | loss  2.83008
| epoch   5 |     1/    0 batches | ms/batch 201.79 | loss  2.40781
| epoch   6 |     1/    0 batches | ms/batch 213.83 | loss  2.27313
| epoch   7 |     1/    0 batches | ms/batch 237.67 | loss  2.26563
| epoch   8 |     1/    0 batches | ms/batch 219.95 | loss  2.07515
| epoch   9 |     1/    0 batches | ms/batch 216.21 | loss  2.00170
| epoch  10 |     1/    0 batches | ms/batch 219.87 | loss  1.95035
| epoch  11 |     1/    0 batches | ms/batch 218.58 | loss  1.74026
| epoch  12 |     1/    0 batches | ms/batch 218.08 | loss  1.77221
| epoch  13 |     1/    0 batches | ms/batch 208.76 | loss  1.78408
| epoch  14 |     1/    0 batches | ms/batch 208.43 | loss  1.71462
| epoch  15 |     1/    0 batches | ms/batch 203

In [14]:
cellGeneCompressed=pickle.load(open(data_path+'cellGeneCompressed.p', 'rb'))
cellGeneCompressed=torch.FloatTensor(cellGeneCompressed)

#Synergy prediction

Here is where the train test splits are created for the model training and evalutation.


##Train/test split in cross or external validation

In [15]:
def get_cell_of_interest(tissues):
    tissues_of_interests = [codes['tissue'].item2idx[minor_tissue] for minor_tissue in tissues]
    cell_of_interest = cell_features.index[cell_features['tissue_id'].isin(tissues_of_interests)].tolist()
    return cell_of_interest

Train/test for general model



In [16]:
minor_tissues=['bone', 'prostate' ]
cell_of_interest = get_cell_of_interest(minor_tissues)
df_tissue_of_interest = df.loc[df['cell_line_name'].isin(cell_of_interest),:]
df_all = df.drop(df_tissue_of_interest.index)
#specific database
#df_all = df_all[df_all['study_id'].isin([1,3])]
df_all = df_all.dropna()
#cross validation
df_train, df_test = train_test_split(df_all, test_size=0.2) #cross validation
#external validation
#df_train=df_all.loc[df_all['study_id']==3] 3: 'ALMANAC'
#df_test=df_all.loc[df_all['study_id']==1] 1: 'ONEIL'

Train/test for bone



In [17]:
_df_bone = df.loc[df['cell_line_name'].isin(get_cell_of_interest(['bone'])),:]
# Cross validation
_df_train_bone, _df_test_bone = train_test_split(_df_bone, test_size=0.2, random_state=1)
# External validation
_df_train_bone=_df_bone.loc[_df_bone['study_id'].isin([7,8,9])]
_df_test_bone= _df_bone.loc[_df_bone['study_id'].isin([7,8,9])]

Train/test for prostate

In [18]:
_df_prostate= df.loc[df['cell_line_name'].isin(get_cell_of_interest(['prostate'])),:]
#Cross validation
_df_train_prostate, _df_test_prostate = train_test_split(_df_prostate, test_size=0.2, random_state=1)
# External validation
_df_train_prostate=_df_prostate.loc[_df_prostate['study_id'].isin([1,3])]
_df_test_prostate=_df_prostate.loc[_df_prostate['study_id'].isin([1,3])]

##Dataset and Dataloader


In [19]:
class DrugCombDataset(Dataset):
    def __init__(self, df, drug_features, cell_features):
        self.df = df
        self.drug_features = drug_features
        self.cell_features = cell_features

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        d1 = self.df.iloc[idx, 0]
        d2 = self.df.iloc[idx, 1]
        cell = self.df.iloc[idx,2]
        ri_d1 = 1.0 if float(self.df.iloc[idx,3]) >ri_threshold else 0
        ri_d2 = 1.0 if float(self.df.iloc[idx,4]) >ri_threshold else 0
        syn = 1.0 if self.df.iloc[idx, 5] >syn_threshold else 0


        #external features
        d1_fp = np.array(self.drug_features.loc[d1, 'fps'])
        d1_sm = self.drug_features.loc[d1, 'smiles']
        d1_sm = np.pad(d1_sm, pad_width=(0, max_drug_sm_len-len(d1_sm)), mode='constant', constant_values=0)
        d1_gn=drugGeneCompressed[d1]

        d2_fp = np.array(self.drug_features.loc[d2, 'fps'])
        d2_sm = self.drug_features.loc[d2, 'smiles']
        d2_sm = np.pad(d2_sm, pad_width=(0, max_drug_sm_len-len(d2_sm)), mode='constant', constant_values=0)
        d2_gn=drugGeneCompressed[d2]


        c_ts = self.cell_features.loc[cell, 'tissue_id']
        c_ds = self.cell_features.loc[cell, 'disease_id']

        c_gn= cellGeneCompressed[cell]

        sample = {
            'd1': d1,
            'd1_fp': d1_fp,
            'd1_sm': d1_sm,
            'd1_gn': d1_gn,

            'd2': d2,
            'd2_fp': d2_fp,
            'd2_sm': d2_sm,
            'd2_gn': d2_gn,

            'cell': cell,
            'c_ts': c_ts,
            'c_ds': c_ds, #missing -1
            'c_gn': c_gn,

            'ri_d1': ri_d1,
            'ri_d2': ri_d2,
            'syn': syn
        }

        return sample


##Load dataset


General model



In [20]:
train = DrugCombDataset(df_train, drug_features, cell_features)
train_loader = DataLoader(train, batch_size=bsz, shuffle=True)
test = DrugCombDataset(df_test, drug_features, cell_features)
test_loader = DataLoader(test, batch_size=bsz, shuffle=True)

Bone

In [21]:
_train_bone = DrugCombDataset(_df_train_bone, drug_features, cell_features)
_train_loader_bone = DataLoader(_train_bone, batch_size=bsz, shuffle=True)
_test_bone = DrugCombDataset(_df_test_bone, drug_features, cell_features)
_test_loader_bone = DataLoader(_test_bone, batch_size=bsz, shuffle=False)

Prostate



In [22]:
_train_prostate = DrugCombDataset(_df_train_prostate, drug_features, cell_features)
_train_loader_prostate = DataLoader(_train_prostate, batch_size=bsz, shuffle=True)
_test_prostate = DrugCombDataset(_df_test_prostate, drug_features, cell_features)
_test_loader_prostate = DataLoader(_test_prostate, batch_size=bsz, shuffle=False)

#Prediction model

Our main training model will be called Comb(). For Comb() to work we need to include 2 other encoder models. These are DrugEncoder() and CellEncoder().

DrugEncoder() loads from the the drug combined dataloader. The comments in the model go over how the model goes through the different parts of the input data in order to prep for the forward pass.

CellEncoder() is much the same but uses the cell lines combined dataloader.

Comb() is the main prediction model. It is made from the drug encoder and the cell encoder models. The forward pass encodes the initial drug, then the starting drug.

The synergy is calculated by concatentation the two drug encoders and also the cell line encoder and passing them through a fully connected layer. The sensitivities for each drug encoding is also calculated by passing them through a fully connected layer after concatentation.

Original Paper Repo: https://github.com/yejinjkim/synergy-transfer/tree/master

In [23]:
class DrugEncoder(nn.Module):
    def __init__(self,
                 num_drugs=num_drugs,
                 num_ID_emb=0,
                 num_drug_fp=num_drug_fp,
                 max_drug_sm_len=max_drug_sm_len,
                 num_gene = num_gene_compressed_drug,
                 num_comp_char=len(codes['mole'].idx2item),
                 fp_embed_sz = 32,
                 gene_embed_sz = int(num_gene_compressed_drug/2),
                 out_size=64,
                 dropout=0.3):
        super(DrugEncoder, self).__init__()

        self.dropout= dropout
        #DRUG
        #drug ID
        #self.embed_id = nn.Embedding(num_drugs, num_ID_emb)

        #compound ID
        self.embed_comp = nn.Embedding(num_comp_char, num_comp_char, padding_idx=0)#padding's idx=0
        #encoding compound
        self.encoderlayer = nn.TransformerEncoderLayer(d_model=num_comp_char, nhead=4)
        self.encoder = nn.TransformerEncoder(self.encoderlayer, num_layers=1)

        #fingerprint
        self.dense_fp = nn.Linear(num_drug_fp,fp_embed_sz)
        #gene
        self.dense_gene = nn.Linear(num_gene,gene_embed_sz)

        #depthwise for compound encoding
        self.conv = nn.Conv2d(1, 1, (1, num_comp_char), groups=1)

        #combined
        combined_sz = num_ID_emb+fp_embed_sz+max_drug_sm_len+gene_embed_sz
        self.FC2 = FC2(combined_sz, out_size, dropout)

    def forward(self, d_list):
        """
            id: bsz*1
            fp: bsz*num_drug_fp
            sm: bsz*max_drug_sm_len
        """
        id, fp, sm, gn = d_list

        sm = self.embed_comp(sm) #bsz*max_drug_sm_len*num_comp_char(embedding size)
        sm = self.encoder(sm)
        sm = self.conv(sm.unsqueeze(1)).squeeze()

        fp = F.relu(self.dense_fp(fp))
        gn = F.relu(self.dense_gene(gn))

        #combine
        x = torch.cat((fp, sm, gn),1) # bsz*[num_emb_id+num_drug_fp+max+drug_sm]
        x = self.FC2(x)

        return x


In [24]:
class CellEncoder(nn.Module):
    def __init__(self,
                 num_cells=num_celllines,
                 num_tissue=0,
                 num_disease=num_disease,
                 num_ID_emb=0,
                 gene_embed_sz=int(num_gene_compressed_cell/2),
                 num_gene=num_gene_compressed_cell,
                 out_size=64,
                 dropout=0.3):
        super(CellEncoder, self).__init__()

        self.dropout= dropout
        #cell ID
        #self.embed_id = nn.Embedding(num_cells, num_ID_emb)
        #cell tissue
        #self.embed_ts = nn.Embedding(num_tissue, num_tissue)
        #cell disease
        self.embed_ds = nn.Embedding(num_disease, num_disease, padding_idx=3)
        #gene
        self.dense_gene = nn.Linear(num_gene,gene_embed_sz)

        #combined
        combined_sz = num_ID_emb+num_tissue+num_disease+gene_embed_sz
        self.FC2 = FC2(combined_sz, out_size, dropout)


    def forward(self, c_list):
        """
            id: bsz*1
            fp: bsz*num_drug_fp
            sm: bsz*max_drug_sm_len
        """
        id, ts, ds, gn = c_list
        ds = F.relu(self.embed_ds(ds)) #bsz*num_diesaes

        gn = F.relu(self.dense_gene(gn)) #bsz*gene_embed_sz

        #combine
        x = torch.cat((ds, gn),1) # bsz*combined_sz
        x = self.FC2(x)

        return x

In [25]:
class Comb(nn.Module):
    def __init__(self, num_cells=num_celllines,
                 num_drugs=num_drugs,
                 num_drug_fp=num_drug_fp,
                 max_drug_sm_len=max_drug_sm_len,
                num_comp_char=len(codes['mole'].idx2item),
                 num_ID_emb=0,
                 out_size=64,
                dropout=0.3):

        super(Comb, self).__init__()

        self.dropout=dropout
        #drug
        self.drugEncoder = DrugEncoder()
        #cell
        self.cellEncoder = CellEncoder()
        #fc
        self.fc_syn = FC2(out_size*3, 1, dropout)
        self.fc_ri = FC2(out_size*2, 1, dropout)

    def forward(self, d1_list, d2_list, c_list):
        d1 = self.drugEncoder(d1_list)
        d2 = self.drugEncoder(d2_list)
        c = self.cellEncoder(c_list)

        syn = self.fc_syn(torch.cat((d1, d2, c),1))
        ri1 = self.fc_ri(torch.cat((d1,c),1))
        ri2 = self.fc_ri(torch.cat((d2,c),1))

        return syn, ri1, ri2

#Training

The model training is as follows. We take our Comb() model and begin training with a Binary Cross Entropy Loss. We use the Adagrad optimizer.

For training we get all the training sample values from the training data loader and then we pass them into the model. The drug synergy is outputted by the model and we compute loss, compute the gradient, and iterate the model.

##Hyperparams
We have a few hyperparameters that we've taken from the paper.
- Our learning rate is the default torch learning rate which is 0.001
- Our batch size is 100
- Our dropout is 0.1

##Computational Requirements
To run this script there are computational requirements.
- We had CUDA set up using 3 devices on Google Colab Pro.
- We trained for 10 epochs
- The average runtime for each epoch was 66 ms per batch and we had 17 batches


In [58]:
model = Comb()
if cuda:
    model = model.cuda()
#Regression
#criterion_mse = nn.MSELoss()
#Classification
criterion_bce = nn.BCEWithLogitsLoss()
optimizer = optim.Adagrad(model.parameters())

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [27]:
#Training
def training(isAux, data_loader):
    model.train()
    total_loss = 0
    start_time = time.time()

    for iteration, sample in enumerate(data_loader):
      try:
        d1=Variable(sample['d1'])
        d1_fp = Variable(sample['d1_fp'].float())
        d1_sm = Variable(sample['d1_sm'])
        d1_gn = Variable(sample['d1_gn'].float())

        d2=Variable(sample['d2'])
        d2_fp = Variable(sample['d2_fp'].float())
        d2_sm = Variable(sample['d2_sm'])
        d2_gn = Variable(sample['d2_gn'].float())

        cell = Variable(sample['cell'])
        c_ts = Variable(sample['c_ts'])
        c_ds = Variable(sample['c_ds'])
        c_gn = Variable(sample['c_gn'].float())

        syn_true = Variable(sample['syn'].float())
        ri_d1=Variable(sample['ri_d1'].float())
        ri_d2=Variable(sample['ri_d2'].float())


        if cuda:
            d1=d1.cuda()
            d1_fp=d1_fp.cuda()
            d1_sm=d1_sm.cuda()
            d1_gn=d1_gn.cuda()

            d2=d2.cuda()
            d2_fp=d2_fp.cuda()
            d2_sm=d2_sm.cuda()
            d2_gn=d2_gn.cuda()

            cell=cell.cuda()
            c_ts=c_ts.cuda()
            c_ds=c_ds.cuda()
            c_gn=c_gn.cuda()

            syn_true=syn_true.cuda()
            ri_d1=ri_d1.cuda()
            ri_d2=ri_d2.cuda()

        optimizer.zero_grad()

        syn, ri1, ri2 = model((d1, d1_fp, d1_sm, d1_gn), (d2, d2_fp, d2_sm, d2_gn), (cell, c_ts, c_ds, c_gn) )

        if not isAux:
            loss = criterion_bce(syn, syn_true.view(-1,1))
        else:
            loss = criterion_bce(ri1, ri_d1.view(-1,1))+criterion_bce(ri2, ri_d2.view(-1,1))

        loss.backward()
        optimizer.step()
        total_loss += loss.data

        if iteration % log_interval == 0 and iteration > 0:
            cur_loss = total_loss.item() / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | ms/batch {:5.2f} | loss {:8.5f}'.format(epoch, iteration, int(len(train_loader)/bsz), elapsed * 1000/log_interval, cur_loss))

            total_loss = 0
            start_time = time.time()
      except:
          continue

In [28]:

def evaluate(data_loader):
    model.eval()
    total_loss = 0
    total_loss_sen = 0


    #loss
    with torch.no_grad():
        for iteration, sample in enumerate(data_loader):
            d1=Variable(sample['d1'])
            d1_fp = Variable(sample['d1_fp'].float())
            d1_sm = Variable(sample['d1_sm'])
            d1_gn = Variable(sample['d1_gn'].float())

            d2=Variable(sample['d2'])
            d2_fp = Variable(sample['d2_fp'].float())
            d2_sm = Variable(sample['d2_sm'])
            d2_gn = Variable(sample['d2_gn'].float())

            cell = Variable(sample['cell'])
            c_ts = Variable(sample['c_ts'])
            c_ds = Variable(sample['c_ds'])
            c_gn = Variable(sample['c_gn'].float())

            syn_true = Variable(sample['syn'].float())
            ri_d1=Variable(sample['ri_d1'].float())
            ri_d2=Variable(sample['ri_d2'].float())


            if cuda:
                d1=d1.cuda()
                d1_fp=d1_fp.cuda()
                d1_sm=d1_sm.cuda()
                d1_gn=d1_gn.cuda()

                d2=d2.cuda()
                d2_fp=d2_fp.cuda()
                d2_sm=d2_sm.cuda()
                d2_gn=d2_gn.cuda()

                cell=cell.cuda()
                c_ts=c_ts.cuda()
                c_ds=c_ds.cuda()
                c_gn=c_gn.cuda()

                syn_true=syn_true.cuda()
                ri_d1=ri_d1.cuda()
                ri_d2=ri_d2.cuda()


            syn,ri1,ri2 = model((d1, d1_fp, d1_sm, d1_gn), (d2, d2_fp, d2_sm, d2_gn), (cell, c_ts,c_ds,c_gn) )
            loss = criterion_bce(syn, syn_true.view(-1,1))
            total_loss +=loss.data
            loss_sen = (criterion_bce(ri1, ri_d1.view(-1,1))+criterion_bce(ri2, ri_d2.view(-1,1)))/2
            total_loss_sen += loss_sen.data

        print('syn mse', total_loss.item()/(iteration+1))
        print('sen_mse', total_loss_sen.item()/(iteration+1))

In [ ]:

best_val_loss = None
try:
    for epoch in range(1, 11):
        epoch_start_time = time.time()
        training(False, train_loader)
        training(True, train_loader)
        evaluate(test_loader)
        print('-'*89)
except KeyboardInterrupt:
    print('-'*89)
    print('Existing from training early')

| epoch   1 |   100/   17 batches | ms/batch 90.82 | loss  0.37345
| epoch   1 |   200/   17 batches | ms/batch 90.16 | loss  0.32039
| epoch   1 |   300/   17 batches | ms/batch 84.83 | loss  0.30158
| epoch   1 |   400/   17 batches | ms/batch 89.44 | loss  0.29340
| epoch   1 |   500/   17 batches | ms/batch 89.39 | loss  0.28483
| epoch   1 |   600/   17 batches | ms/batch 86.29 | loss  0.29483
| epoch   1 |   700/   17 batches | ms/batch 89.03 | loss  0.27432
| epoch   1 |   800/   17 batches | ms/batch 88.85 | loss  0.27499
| epoch   1 |   900/   17 batches | ms/batch 84.80 | loss  0.27606
| epoch   1 |  1000/   17 batches | ms/batch 90.83 | loss  0.25913
| epoch   1 |  1100/   17 batches | ms/batch 106.84 | loss  0.27614
| epoch   1 |  1200/   17 batches | ms/batch 89.53 | loss  0.26944
| epoch   1 |  1300/   17 batches | ms/batch 88.52 | loss  0.26507
| epoch   1 |  1400/   17 batches | ms/batch 87.24 | loss  0.25955
| epoch   1 |  1500/   17 batches | ms/batch 85.31 | loss  0.

In [71]:
#model save
#torch.save(model.state_dict(), data_path+'15-Pooled.p')
#model load
model.load_state_dict(torch.load(data_path+'15-Pooled.p'))
#model.eval()

<All keys matched successfully>

#Evaluate the general model
The model is evaluated based on a few metrics the original paper uses the metrics as: AUROC, AUPRC

The original paper achieved these results:

{0.9577 AUROC, 0.8335 AUPRC} (General Model)

Our implementation for this project will aim to hit these metrics. Any ablations we will also aim to hit these metrics.

## Achieved Results

In [30]:
def dcg_score(y_score, y_true, k):
    """
        https://www.kaggle.com/davidgasquez/ndcg-scorer
        y_true: np.array, size= [n_samples]
        y_score: np.array, size=[n_samples]
        k: int, rank
    """
    order = np.argsort(y_score)[::-1]
    y_true = np.take(y_true, order[:k])

    #gain = 2 ** y_true -1
    gain = y_true

    discounts = np.log2(np.arange(len(y_true)) + 2)
    return np.sum(gain/discounts)

def evaluate_accuracy(data_loader):
    model.eval()

    syn_all=[]
    syn_true_all=[]
    ri1_all=[]
    ri1_true_all=[]
    ri2_all=[]
    ri2_true_all=[]

    #loss
    with torch.no_grad():
        for iteration, sample in enumerate(data_loader):
            d1=Variable(sample['d1'])
            d1_fp = Variable(sample['d1_fp'].float())
            d1_sm = Variable(sample['d1_sm'])
            d1_gn = Variable(sample['d1_gn'].float())

            d2=Variable(sample['d2'])
            d2_fp = Variable(sample['d2_fp'].float())
            d2_sm = Variable(sample['d2_sm'])
            d2_gn = Variable(sample['d2_gn'].float())

            cell = Variable(sample['cell'])
            c_ts = Variable(sample['c_ts'])
            c_ds = Variable(sample['c_ds'])
            c_gn = Variable(sample['c_gn'].float())

            syn_true = Variable(sample['syn'].float())
            ri_d1=Variable(sample['ri_d1'])
            ri_d2=Variable(sample['ri_d2'])

            if cuda:
                d1=d1.cuda()
                d1_fp=d1_fp.cuda()
                d1_sm=d1_sm.cuda()
                d1_gn=d1_gn.cuda()

                d2=d2.cuda()
                d2_fp=d2_fp.cuda()
                d2_sm=d2_sm.cuda()
                d2_gn=d2_gn.cuda()

                cell=cell.cuda()
                c_ts=c_ts.cuda()
                c_ds=c_ds.cuda()
                c_gn=c_gn.cuda()


            syn,ri1,ri2 = model((d1, d1_fp, d1_sm, d1_gn), (d2, d2_fp, d2_sm, d2_gn), (cell, c_ts,c_ds,c_gn) )

            syn_all.append(syn.data.cpu().numpy())
            syn_true_all.append(syn_true.numpy())

            ri1_all.append(ri1.data.cpu().numpy())
            ri1_true_all.append(ri_d1.numpy())

            ri2_all.append(ri2.data.cpu().numpy())
            ri2_true_all.append(ri_d2.numpy())

    return syn_all, syn_true_all, ri1_all, ri1_true_all, ri2_all, ri2_true_all

## Evaluate synergy prediction (General Model)




In [73]:
syn_all, syn_true_all, ri1_all, ri1_true_all, ri2_all, ri2_true_all= evaluate_accuracy(test_loader)

syn_all= [s.item() for syn in syn_all for s in syn]
syn_true_all = [s for syn in syn_true_all for s in syn]
#NDCG
print(dcg_score(syn_all,syn_true_all, k=20)/dcg_score(syn_true_all,syn_true_all, k=20))
#AUPRC
print(metrics.average_precision_score(syn_true_all,  1/(1 + np.exp(-np.array(syn_all)))))
#AUROC
print(metrics.roc_auc_score(syn_true_all,  1/(1 + np.exp(-np.array(syn_all)))))

1.0
0.7267833130011351
0.9257503044782515


## Evaluate sensitivity prediction (General)



In [74]:

ri1_all= [r.item() for ri in ri1_all for r in ri]
ri1_true_all = [r for ri in ri1_true_all for r in ri]

ri2_all= [r.item() for ri in ri2_all for r in ri]
ri2_true_all = [r for ri in ri2_true_all for r in ri]

ri_all=ri1_all+ri2_all
ri_true_all=ri1_true_all+ri2_true_all
#NDCG
dcg_score(ri_all,ri_true_all, k=20)/dcg_score(ri_true_all,ri_true_all, k=20)
#AUC
metrics.roc_auc_score(ri_true_all,  1/(1 + np.exp(-np.array(ri_all))))

0.8462647050183483

#Transfer the general model to specific model


If you want to boost a bit more with general model's test set



In [ ]:
#Use major's test set
training(False, test_loader)
training(True, test_loader)

| epoch  10 |   100/   17 batches | ms/batch 87.88 | loss  0.23159
| epoch  10 |   200/   17 batches | ms/batch 87.01 | loss  0.20515
| epoch  10 |   300/   17 batches | ms/batch 85.83 | loss  0.21154
| epoch  10 |   400/   17 batches | ms/batch 89.06 | loss  0.20682
| epoch  10 |   500/   17 batches | ms/batch 88.33 | loss  0.19914
| epoch  10 |   100/   17 batches | ms/batch 89.85 | loss  0.16814
| epoch  10 |   200/   17 batches | ms/batch 85.97 | loss  0.16068
| epoch  10 |   300/   17 batches | ms/batch 88.43 | loss  0.16447
| epoch  10 |   400/   17 batches | ms/batch 88.77 | loss  0.15749
| epoch  10 |   500/   17 batches | ms/batch 84.70 | loss  0.15284


##Freeze layers

Examine the layer's ID that we'd like to fix or free

In [ ]:
for i, param in enumerate(model.parameters()):
    print(i, param.size(), param.requires_grad)
release_after = 46
for i, param in enumerate(model.parameters()):
    if i>=release_after:
        param.requires_grad=True
    else:
        param.requires_grad=False

0 torch.Size([52, 52]) True
1 torch.Size([156, 52]) True
2 torch.Size([156]) True
3 torch.Size([52, 52]) True
4 torch.Size([52]) True
5 torch.Size([2048, 52]) True
6 torch.Size([2048]) True
7 torch.Size([52, 2048]) True
8 torch.Size([52]) True
9 torch.Size([52]) True
10 torch.Size([52]) True
11 torch.Size([52]) True
12 torch.Size([52]) True
13 torch.Size([156, 52]) True
14 torch.Size([156]) True
15 torch.Size([52, 52]) True
16 torch.Size([52]) True
17 torch.Size([2048, 52]) True
18 torch.Size([2048]) True
19 torch.Size([52, 2048]) True
20 torch.Size([52]) True
21 torch.Size([52]) True
22 torch.Size([52]) True
23 torch.Size([52]) True
24 torch.Size([52]) True
25 torch.Size([32, 167]) True
26 torch.Size([32]) True
27 torch.Size([32, 64]) True
28 torch.Size([32]) True
29 torch.Size([1, 1, 1, 52]) True
30 torch.Size([1]) True
31 torch.Size([352]) True
32 torch.Size([352]) True
33 torch.Size([176, 352]) True
34 torch.Size([176]) True
35 torch.Size([64, 176]) True
36 torch.Size([64]) True
37

Prostate or bone



In [40]:
_train_loader_minor=_train_loader_prostate
_test_loader_minor=_test_loader_prostate
_test_minor=_test_prostate
#_train_loader_minor=_train_loader_bone
#_test_loader_minor=_test_loader_bone
#_test_minor=_test_bone

In [48]:
#Use minor's train set
try:
    for epoch in range(1, epochs+1):
        epoch_start_time = time.time()
        training(False, _train_loader_minor)
        training(True, _train_loader_minor)
        evaluate(_test_loader_minor)
        print('-'*89)
except KeyboardInterrupt:
    print('-'*89)
    print('Existing from training early')

syn mse 0.18647881392594223
sen_mse 0.05407539304796156
-----------------------------------------------------------------------------------------
syn mse 0.13272172278100317
sen_mse 0.05336044122884562
-----------------------------------------------------------------------------------------
syn mse 0.12024202451601133
sen_mse 0.05167775625710959
-----------------------------------------------------------------------------------------
syn mse 0.11358097621372767
sen_mse 0.05193812506539481
-----------------------------------------------------------------------------------------
syn mse 0.10983221871512276
sen_mse 0.051680072323306576
-----------------------------------------------------------------------------------------
syn mse 0.10576765877859932
sen_mse 0.05199513592562833
-----------------------------------------------------------------------------------------
syn mse 0.10389151939978966
sen_mse 0.05193541600153996
-------------------------------------------------------------------

## Evaluate synergy prediction (No transfer Prostate)


In [75]:
model.load_state_dict(torch.load(data_path+'15-Pooled-no-transfer-prostate.p'))

<All keys matched successfully>

In [76]:
syn_all, syn_true_all, ri1_all, ri1_true_all, ri2_all, ri2_true_all= evaluate_accuracy(_test_loader_minor)

syn_all= [s.item() for syn in syn_all for s in syn]
syn_true_all = [s for syn in syn_true_all for s in syn]
#NDCG
print(dcg_score(syn_all,syn_true_all, k=20)/dcg_score(syn_true_all,syn_true_all, k=20))
#AUROC
print(metrics.roc_auc_score(syn_true_all,  1/(1 + np.exp(-np.array(syn_all)))))
#AUPRC
print(metrics.average_precision_score(syn_true_all,  1/(1 + np.exp(-np.array(syn_all)))))

1.0
0.986630348869761
0.9114650739035156


## Evaluate sensitivity prediction (No transfer Prostate)


In [77]:
ri1_all= [r.item() for ri in ri1_all for r in ri]
ri1_true_all = [r for ri in ri1_true_all for r in ri]

ri2_all= [r.item() for ri in ri2_all for r in ri]
ri2_true_all = [r for ri in ri2_true_all for r in ri]

ri_all=ri1_all+ri2_all
ri_true_all=ri1_true_all+ri2_true_all
#NDCG
dcg_score(ri_all,ri_true_all, k=20)/dcg_score(ri_true_all,ri_true_all, k=20)
#AUC
metrics.roc_auc_score(ri_true_all,  1/(1 + np.exp(-np.array(ri_all))))

0.7897974169720681

## Evaluate synergy prediction (Transfer Retrain All Parameters Prostate)



In [78]:
model.load_state_dict(torch.load(data_path+'15-Pooled-transfer-retrain-prostate.p'))

<All keys matched successfully>

In [79]:
syn_all, syn_true_all, ri1_all, ri1_true_all, ri2_all, ri2_true_all= evaluate_accuracy(_test_loader_minor)

syn_all= [s.item() for syn in syn_all for s in syn]
syn_true_all = [s for syn in syn_true_all for s in syn]
#NDCG
print(dcg_score(syn_all,syn_true_all, k=20)/dcg_score(syn_true_all,syn_true_all, k=20))
#AUROC
print(metrics.roc_auc_score(syn_true_all,  1/(1 + np.exp(-np.array(syn_all)))))
#AUPRC
print(metrics.average_precision_score(syn_true_all,  1/(1 + np.exp(-np.array(syn_all)))))

1.0
0.9969655452679639
0.9813636350945647


## Evaluate sensitivity prediction (Transfer Retrain All Parameters Prostate)



In [80]:
ri1_all= [r.item() for ri in ri1_all for r in ri]
ri1_true_all = [r for ri in ri1_true_all for r in ri]

ri2_all= [r.item() for ri in ri2_all for r in ri]
ri2_true_all = [r for ri in ri2_true_all for r in ri]

ri_all=ri1_all+ri2_all
ri_true_all=ri1_true_all+ri2_true_all
#NDCG
dcg_score(ri_all,ri_true_all, k=20)/dcg_score(ri_true_all,ri_true_all, k=20)
#AUC
metrics.roc_auc_score(ri_true_all,  1/(1 + np.exp(-np.array(ri_all))))

0.8036510078963031

#Select the top ranked drug combinations


In [ ]:
syn_all_prob=1/(1 + np.exp(-np.array(syn_all)))
order = np.argsort(syn_all_prob)[::-1]
syn_true_all_order = np.take(syn_true_all, order[:20])
for k in range(20):
    comb=_test_minor[order[k]]
    print(codes['drugs'].idx2item[comb['d1']], ',',
          codes['drugs'].idx2item[comb['d2']], ',',
          codes['cell'].idx2item[comb['cell']],  ',',
          codes['tissue'].idx2item[comb['c_ts']],  ',',
          codes['disease'].idx2item[comb['c_ds']], ',',
          comb['ri_d1'], ',',
          comb['ri_d2'], ',',
          syn_true_all_order[k])

5-Fluoro-2'-deoxyuridine , Trisenox , DU-145 , prostate , prostate , 0 , 0 , 1.0
cyclophosphamide , Trisenox , DU-145 , prostate , prostate , 0 , 0 , 1.0
Nilotinib , Trisenox , DU-145 , prostate , prostate , 0 , 0 , 1.0
Pralatrexate , Trisenox , DU-145 , prostate , prostate , 0 , 0 , 1.0
Procarbazine hydrochloride , Trisenox , DU-145 , prostate , prostate , 0 , 0 , 1.0
6-Mercaptopurine , Trisenox , DU-145 , prostate , prostate , 0 , 0 , 1.0
5-Fluoro-2'-deoxyuridine , Trisenox , PC-3 , prostate , prostate , 0 , 0 , 1.0
MK-2206 , topotecan , LNCAP , prostate , prostate , 0 , 1.0 , 1.0
Bortezomib , Trisenox , DU-145 , prostate , prostate , 0 , 0 , 1.0
122111-05-1 , Trisenox , DU-145 , prostate , prostate , 0 , 0 , 1.0
Eloxatin (TN) (Sanofi Synthelab) , Trisenox , DU-145 , prostate , prostate , 0 , 0 , 1.0
Antibiotic AY 22989 , Trisenox , DU-145 , prostate , prostate , 0 , 0 , 1.0
Nilotinib , Trisenox , PC-3 , prostate , prostate , 0 , 0 , 1.0
6-Mercaptopurine , Ixabepilone , PC-3 , prosta

# Results

This section will go over the results of the project in comparison to the results of the paper.

| Model    | AUROC    | AUPRC    |
|----------|----------|----------|
|   Original General  |   .9577  |   .8335  |
|   Reproduced General  |   .9258  |   .7270  |
|   Original Prostate No Transfer (ALMANC,ONEIL)  |   .9775  |   .8575  |
|   Reproduced Prostate No Transfer (ALMANC,ONEIL)  |   .9866  |   .9115  |
|   Original Prostate Transfer Retrain (ALMANC,ONEIL)  |   .9928  |   .9628  |
|   Reproduced Prostate Transfer Retrain (ALMANC,ONEIL)  |   .9970 |   .9814  |

As shown by the table we have achieved similar results with the ALMANAC and ONEIL datasets. The original paper hypothesis was given data rich tissue we can use transfer learning on the model in order to imporve the results of drug synergy prediction. The orginal paper showed that with an increase in the AUROC and AUPRC scores. We have also shown an increase in both of those metrics after the transfer learning. Therefore this shows as evidence that transfer learning can be applied to drug synergy prediction models.

The goal of the paper was to train a general model and use whatever data was availble for a data poor tissue and use transfer learning to make a drug synergy prediction model for that data poor tissue. We have shown strong evidence that supports the hypothesis of the orginal paper. With these evaluation metrics we have proof that our model can perform effective drug synergy prediction on prostate tissue which is data poor.

## Ablation Study

We plan to remove the dropout layer for the model. We believe that over-fitting is not necessarily possible with this scenario as this model can be trained and targeted to different forms of human body tissue. Therefore we do not need to worry about this being a general model in totality, we can tailor it to one tissue at a time.

## Ablation Results

When performing this ablation without a dropout in the encoder models we found that the model became overfitted to the pooled databases. To retiterate, because the data pool was varied in the tissues included we believe that the encoding for the drug and tissues would represent the data well. However we observed that the model did not perform as well at all.

Our results for the general model appear to be roughly the same, our AUROC was .9356 and our AUPRC was .8122. This appeared to be fine but after transfer learning we saw a decrease in results. We tested the transfer learning in the same manner the general model was transfered and retrained and retested again prostate tissue. We found that our AUROC was .7148 and our AUPRC was .6520. This is roughly 30% decrease in score from the original papers results.

The takeaway is that the dropout is a necessary hypermparameter to the performance of the model. We suspect that because the data rich tissue is multiple times more abundant than the data poor datasets overfitting is all but guaranteed to happen. Transfer learning relies on the model being general enough such that new hyperparameters can be trained. However, we see that the general model requires the dropout to not be overfitted.

# Discussion
The original paper was able to be somewhat reproduced. The paper used many databases and applied different learning methods to obtain drug synergy score.
We were able to use and demonstrate a few databases namely those using prostate tissue. We were able to achieve results close enough to the original paper results. There was some discrepancy which will be discussed further.

Predominantly the paper was reproducible and with more time, we may have achieved the exact same results of the original author. Using Google Colab we had all the hardware we need in order to run the code and evaluate the model. All the data was accessicible and even the author was able to be reached via communication. The source code was up to date and fit into modern python and programming standards, and the github was public. For the purpose of reproduction this paper had all the fundamentals to be reproducible.

However there are some things that could not be reproduced given the time spent on the project. We could not use all the datasets used to train different models on different tissue. The reason for this is because the datasets provided and the code expectation of the input were not synchronized. There were elements in the input that the code looked for that were not in the datasets themselves. Furthermore, there was an expectation that the reader would know where to load in the data, and where to make changes, in order to run different datasets. This was not clear however, and would take a lot of experience in the deep learning for healthcare field for it to be recognizable.

One of the easiest parts was acquiring the data and building the preprocessing file. The preprocessing file was easy because the data that we got from the author slipped fairly easily into the provided code. Using the graphs we were able to understand and report on the data. Another part that was fairly easy was running the given code. There was minimal editing required on our part once the data had undergone the necessary preprocessing.

One of the difficult parts was using the data in computation. The lack of clarity on how to use the different datasets was a large challenge. There was also some additional preprocessing on our end that needed to be done. For example disease name and tissue name columns were missing in the cell file. We had to incorporate these through some adhoc method.

To improve the reproducibility we would ask the authors to add more information on how to use the data in various ways, and tell the readers what parts of the dataset they are targetting. Another possible improvement would be storing versioned files. The current drugcomb file we used is several versions ahead of the one the paper used. Having the original version would have helped compare results more accurately.

#References

1.   Jaeger, S., Duran-Frigola, M. & Aloy, P. Drug sensitivity in cancer cell lines is not tissue-specific.
Mol Cancer 14, 40 (2015).
https://doi-org.proxy2.library.illinois.edu/10.1186/s12943-015-0312-6
2.   Kim, Y., Zheng, S., Tang, J., Jim Zheng, W., Li, Z., & Jiang, X. (2021). Anticancer drug synergy
prediction in understudied tissues using transfer learning. Journal of the American
Medical Informatics Association : JAMIA, 28(1), 42–51.
https://doi.org/10.1093/jamia/ocaa212
3. Yao F, Madani Tonekaboni SA, Safikhani Z, Smirnov P, El-Hachem N, Freeman M, Manem VSK,
Haibe-Kains B. Tissue specificity of in vitro drug sensitivity. J Am Med Inform Assoc.
2018 Feb 1;25(2):158-166. doi: 10.1093/jamia/ocx062. PMID: 29016819; PMCID:
PMC6381764.
4. Zheng, S., Aldahdooh, J., Shadbahr, T., Wang, Y., Aldahdooh, D., Bao, J., Wang, W., & Tang, J.
(2021). Drugcomb update: A more comprehensive drug sensitivity data repository and
Analysis Portal. Nucleic Acids Research, 49(W1). https://doi.org/10.1093/nar/gkab438
5. Kim, Y. (2024). Genetic Features, https://drive.google.com/drive/folders/1Dg8X_R6-qyfjEtgAYGw7RAo94-6JhfNp?usp=sharing